# import required libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import json
from keras.models import Sequential
from keras.layers import Conv2D , MaxPooling2D , Flatten ,Dense ,Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Precision, Recall, TopKCategoricalAccuracy

# Setup Environment

In [2]:
img_size= (224,224)
input_shape_3D = (224,224,3)
seed = 1
batch_size = 64
epochs = 30

# Load Original Image data

In [3]:
directory="D:/rice_disease_detection/dataset/Original_dataset"

In [4]:
original_data = tf.keras.utils.image_dataset_from_directory(directory=directory,
                                                  labels='inferred',
                                                  label_mode='categorical',
                                                  class_names=None ,
                                                  color_mode='rgb',
                                                  image_size=img_size,
                                                  seed=seed,
                                                  validation_split=0.3,
                                                  subset='training',
                                                  )

Found 25144 files belonging to 10 classes.
Using 17601 files for training.


In [1]:
original_val_data = tf.keras.utils.image_dataset_from_directory(directory=directory,
                                                  labels='inferred',
                                                  label_mode='categorical',
                                                  class_names=None ,
                                                  color_mode='rgb',
                                                  image_size=img_size,
                                                  seed=seed,
                                                  validation_split=0.3,
                                                  subset='validation',
                                                  )

NameError: name 'tf' is not defined

# Rescale Images

In [13]:
# Apply rescaling using the Rescaling layer
train_data=train_data.map(lambda x, y: (tf.keras.layers.Rescaling(1./255)(x), y))
val_data=val_data.map(lambda x, y: (tf.keras.layers.Rescaling(1./255)(x), y))

In [14]:
# Apply caching and prefetching
train_data = train_data.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
val_data = val_data.cache().prefetch(buffer_size=tf.data.AUTOTUNE)


# Create Model

In [15]:
# Define Model

model = Sequential()
model.add(Conv2D(filters=16 ,kernel_size=3 ,padding='same' ,strides=1 ,activation='relu' ,use_bias=False ,input_shape=input_shape_3D))
model.add(MaxPooling2D(pool_size=(3,3)))

model.add(Conv2D(filters=32 ,kernel_size=3 ,padding='same' ,strides=1 ,activation='relu' ,use_bias=False))
model.add(MaxPooling2D(pool_size=(3,3)))

model.add(Conv2D(filters=64 ,kernel_size=3 ,padding='same' ,strides=1 ,activation='relu' ,use_bias=False))
model.add(MaxPooling2D(pool_size=(3,3)))

model.add(Conv2D(filters=128 , kernel_size=3 ,padding='same' ,strides=1 ,activation='relu' ,use_bias=False))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=256 , kernel_size=3 ,padding='same' ,strides=1 ,activation='relu' ,use_bias=False))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(250,activation='relu'))
model.add(Dense(100,activation='relu'))
model.add(Dense(10,activation='softmax'))


model.summary()

d:\rice_disease_detection\rice_disease_env\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 224, 224, 16)   │           432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 74, 74, 32)     │         4,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 24, 24, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 24, 24, 64)     │        18,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 8, 8, 128)      │        73,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 4, 4, 256)      │       294,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 2, 2, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 2, 2, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 250)            │       256,250 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 100)            │        25,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │         1,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 674,472 (2.57 MB)

 Trainable params: 674,472 (2.57 MB)

 Non-trainable params: 0 (0.00 B)

# Train Model

In [16]:
# Define the model with optimizer , loss , metrics
model.compile(optimizer='Adam',loss = 'categorical_crossentropy', metrics=['accuracy', Precision(), Recall(), TopKCategoricalAccuracy(k=3)])

In [17]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

: 

In [18]:
history = model.fit(train_data,
                    epochs=2,
                    validation_data=val_data,
                    callbacks=[early_stopping],
                    verbose=1,
                    shuffle=True)

Epoch 1/2
 736/3878 ━━━━━━━━━━━━━━━━━━━━ 1:24:33 2s/step - accuracy: 0.2864 - loss: 1.9473 - precision: 0.5490 - recall: 0.0732 - top_k_categorical_accuracy: 0.5793

# Evaluate Model's Performance

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(18, 10))

# Plot Accuracy
plt.subplot(2, 3, 1)
plt.plot(history.history['accuracy'], label='Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel("No. of Epochs")
plt.ylabel("Accuracy")
plt.legend(loc="lower right")
plt.title("Accuracy over Epochs")

# Plot Loss
plt.subplot(2, 3, 2)
plt.plot(history.history['loss'], label='Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel("No. of Epochs")
plt.ylabel("Loss")
plt.legend(loc="upper right")
plt.title("Loss over Epochs")

# Plot Precision
plt.subplot(2, 3, 3)
plt.plot(history.history['precision_3'], label='Precision')
plt.plot(history.history['val_precision_3'], label='Validation Precision')
plt.xlabel("No. of Epochs")
plt.ylabel("Precision")
plt.legend(loc="lower right")
plt.title("Precision over Epochs")

# Plot Recall
plt.subplot(2, 3, 4)
plt.plot(history.history['recall_3'], label='Recall')
plt.plot(history.history['val_recall_3'], label='Validation Recall')
plt.xlabel("No. of Epochs")
plt.ylabel("Recall")
plt.legend(loc="lower right")
plt.title("Recall over Epochs")

# Plot Top-k Categorical Accuracy
plt.subplot(2, 3, 5)
plt.plot(history.history['top_k_categorical_accuracy'], label='Top-1 Accuracy')
plt.plot(history.history['val_top_k_categorical_accuracy'], label='Validation Top-1 Accuracy')
plt.xlabel("No. of Epochs")
plt.ylabel("Top-1 Accuracy")
plt.legend(loc="lower right")
plt.title("Top-1 Accuracy over Epochs")

plt.tight_layout()
plt.show()



In [ ]:
# print Accuracy

test_Accuracy = model.evaluate(val_data)
print(f"Model's Accuracy : {test_Accuracy[1]*100}")

# Best Wieghts

In [ ]:
best_epoch = history.history['val_accuracy'].index(max(history.history['val_accuracy'])) + 1
best_weights = model.get_weights()

print(f"Best weights were obtained at epoch {best_epoch}")
print("Best Weights:", best_weights)

# Checking Labels

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))

# Iterate over the batches of images and labels from train_data
for images, labels in train_data.take(1):
    for i in range(25):
        plt.subplot(5, 5, i + 1)
        plt.imshow((images[i].numpy() * 255).astype('uint8'))

        # Convert one-hot encoded label to integer index
        label_index = np.argmax(labels[i].numpy())
        plt.title(class_names[label_index])

        plt.axis('off')

    plt.tight_layout()
    plt.show()


In [ ]:
class_indices = {index: name for index, name in enumerate(class_names)}

In [ ]:
with open('class_indices.json', 'w') as f:
    json.dump(class_indices, f)

In [ ]:
model.save("model/rice_model_new-trial.h5")